In [ ]:
import pandas as pd
import numpy as np


In [ ]:
data=pd.read_csv('C:/Users/DC/Downloads/happiness.csv')
data

In [ ]:
df=pd.DataFrame(data)

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df['Country'].count

In [ ]:
from scipy.stats import zscore
threhold=3
z=np.abs(zscore(df.iloc[:,2:12]))
z

In [ ]:
print(np.where(z>3))

In [ ]:
df_new=df[(z<3).all(axis=1)]

In [ ]:
df_new

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_new.skew()

In [ ]:
df_eda=df_new.drop(['Country'],axis=1)

In [ ]:
df_eda.drop(['Region'],axis=1,inplace=True)

In [ ]:
df_eda['Region'].unique

In [ ]:
df_new['Happiness Score'].plot(kind='hist')

In [ ]:
df_new['Economy (GDP per Capita)'].plot(kind='hist')

In [ ]:
df_new['Family'].plot(kind='hist')

In [ ]:
df_new['Health (Life Expectancy)'].plot(kind='hist')

In [ ]:
df_new['Freedom'].plot(kind='hist')

In [ ]:
df_new['Trust (Government Corruption)'].plot(kind='hist')

In [ ]:
df_new['Generosity'].plot(kind='hist')

In [ ]:
for i in df_eda.columns:
    plt.figure()
    sns.scatterplot(i,'Happiness Score',data=df_eda)

In [ ]:
sns.pairplot(df_eda)

In [ ]:
for i in df_eda.columns:
    plt.figure()
    sns.lineplot(i,'Happiness Score',data=df_eda)

It can be seen that:
    Happines Score increases with higher generosity,trust,freedom,health,family,economy

In [ ]:
!pip install folium

In [ ]:
import folium

In [ ]:
world_map=folium.Map(zoom_start=2,titles='Mapbox Bright')

In [ ]:
world_geo=r'world_countries.json'

In [ ]:
world_map.choropleth(geo_path=world_geo,geo_data=df_new,columns=(['Country'],['Happiness Score']),key_on='feature.properties.name',fill_color='YlDrRd',legend_name='Happiness index')

In [ ]:
world_map

In [ ]:
df_new.corr()

In [ ]:
plt.figure()
sns.heatmap(df_new.corr(),annot=True)

In [ ]:
x=df_new.drop(['Happiness Score','Country','Region'],axis=1)
y=df_new['Happiness Score']
x

In [ ]:
y=y.values.reshape(-1,1)
y.shape

Happiness score has high positive correlation with Economy,family and Health

In [ ]:
def max_score(model):
    max_score=0
    for r_state in range(42,100):
        xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=r_state,test_size=0.20)
        model.fit(xtrain,ytrain)
        ypred=model.predict(xtest)
        r2=r2_score(ytest,ypred)
        print('r2 score is',r2,'with r state',r_state)
        if r2>max_score:
            max_score=r2
            final_r_state=r_state
            print(max_score,'is max accuracy against r_state',final_r_state)
            print(cross_val_score(model,x,y,cv=10).mean())

In [ ]:
max_score(LinearRegression())

The best r2 score comes with random state 72 and cv value is shows no overfitting or underfitting

In [ ]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=72,test_size=0.20)
lr=LinearRegression()
lr.fit(xtrain,ytrain)
ypred=lr.predict(xtest)
r2=r2_score(ytest,ypred)
print('R2 Score:',r2)
print('Mean absolute error:', mean_absolute_error(ytest,ypred))
print('Mean Squared erroe:', mean_squared_error(ytest,ypred))
print('Root Mean Sqaured Error:', np.sqrt(mean_squared_error(ytest,ypred)))
print('Cross Validation Score:',cross_val_score(lr,x,y,cv=10).mean()) 

the linear model performs very well

In [110]:
joblib.dump(lr,'lr_happy.obj')

['lr_happy.obj']